# Atividade de PDI - 1
## Tema: Segmentação de regiões

Realizar experimentos de processamento de imagens para segmentar uma região de interesse na imagem.
são solicitados as seguintes operações:

i - Extrair a componente "RED" da figura flower;

ii - Aplicar uma função para binarizar a componente RED: parâmetros de entrada (imagem RED e threshold);

iii - Aplicar uma função para detectar regiões de conexão na imagem de entrada: parâmetros (imagem RED e coordenada de referência);

iv - Detectar dimensões na região identificada na função anterior utilizando as diferentes medidas de distância: De, D4, D8;


In [1]:
# importando as bibliotecas

import cv2
import numpy as np
import math

C:\Users\danil\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\danil\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\danil\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [24]:
# função de threshold
def threshold(img, threshold):
    imgOut = np.zeros(np.shape(img))
    for i in range(0, np.shape(img)[0]):
        for j in range(0, np.shape(img)[1]):
            if img[i,j]>threshold:
                imgOut[i,j] = 1
    return imgOut

# função para calcular as medidas de largura e altura
def metricsCalc(img, dist):
    width, height = 0, 0
    ph1, ph2, pw1, pw2 = (0,0), (0,0), (0,0), (0,0)
    
    # obtem pontos máximo e mínimo da altura
    for i in range(0, np.shape(img)[0]):
        positions_j = np.where(img[i,:] == 1)
        if positions_j[0].size > 0:
            ph1 = (positions_j[0][0], i)
            break
            
    for i in range(np.shape(img)[0]-1, 0, -1):
        positions_j = np.where(img[i,:] == 1)
        if positions_j[0].size > 0:
            ph2 = (positions_j[0][0], i)
            break
    
    # obtem pontos máximo e mínimo da largura
    for j in range(0, np.shape(img)[1]):
        positions_i = np.where(img[:,j] == 1)
        if positions_i[0].size > 0:
            pw1 = (j, positions_i[0][0])
            break
            
    for j in range(np.shape(img)[1]-1, 0, -1):
        positions_i = np.where(img[:,j] == 1)
        if positions_i[0].size > 0:
            pw2 = (j, positions_i[0][0])
            break
               
    if dist=="DE":   
        height = math.sqrt((ph2[0] - ph1[0])**2 + (ph2[1] - ph1[1])**2)
        width  = math.sqrt((pw2[0] - pw1[0])**2 + (pw2[1] - pw1[1])**2)
       
    elif dist=="D4":
        height = abs(ph2[0] - ph1[0]) + abs(ph2[1] - ph1[1])
        width = abs(pw2[0] - pw1[0]) + abs(pw2[1] - pw1[1])
        
    elif dist=="D8":
        height = max(abs(ph2[0] - ph1[0]), abs(ph2[1] - ph1[1]))
        width =  max(abs(pw2[0] - pw1[0]), abs(pw2[1] - pw1[1]))
        
    return width, height

# função de adjacência-4
def adj4(i, j, img,newPixels):
    if img[i,j-1] == 1 and imgOut[i,j-1] == 0:
        imgOut[i,j-1] = 1
        newPixels = newPixels + 1
    if img[i,j+1] == 1 and imgOut[i,j+1] == 0:
        imgOut[i,j+1] = 1
        newPixels = newPixels + 1
    if img[i-1,j] == 1 and imgOut[i-1,j] == 0:
        imgOut[i-1,j] = 1
        newPixels = newPixels + 1
    if img[i+1,j] == 1 and imgOut[i+1,j] == 0:
        imgOut[i+1,j] = 1
        newPixels = newPixels + 1
    return newPixels

# função de adjacência-8
def adj8(i, j, img,newPixels):
    if img[i,j-1] == 1 and imgOut[i,j-1] == 0:
        imgOut[i,j-1] = 1
        newPixels = newPixels + 1
    if img[i,j+1] == 1 and imgOut[i,j+1] == 0:
        imgOut[i,j+1] = 1
        newPixels = newPixels + 1
    if img[i-1,j] == 1 and imgOut[i-1,j] == 0:
        imgOut[i-1,j] = 1
        newPixels = newPixels + 1
    if img[i+1,j] == 1 and imgOut[i+1,j] == 0:
        imgOut[i+1,j] = 1
        newPixels = newPixels + 1
    if img[i-1,j+1] == 1 and imgOut[i-1,j+1] == 0:
        imgOut[i-1,j+1] = 1
        newPixels = newPixels + 1
    if img[i-1,j-1] == 1 and imgOut[i-1,j-1] == 0:
        imgOut[i-1,j-1] = 1
        newPixels = newPixels + 1
    if img[i+1,j-1] == 1 and imgOut[i+1,j-1] == 0:
        imgOut[i+1,j-1] = 1
        newPixels = newPixels + 1
    if img[i+1,j+1] == 1 and imgOut[i+1,j+1] == 0:
        imgOut[i+1,j+1] = 1
        newPixels = newPixels + 1
    return newPixels

#intercessão de conjuntos de vizinhança-4
def interc(i1, j1, i2, j2, img, newPixels):
    conj1 = np.array([img[i1-1,j1], img[i1,j1-1], img[i1+1,j1], img[i1,j1+1]])
    conj2 = np.array([img[i2-1,j2], img[i2,j2-1], img[i2+1,j2], img[i2,j2+1]])
    vals_c1 = np.where(conj1[:] == 1)
    vals_c2 = np.where(conj2[:] == 1)
    
    if(vals_c1[0].size == 0 or  vals_c2[0].size == 0):
        imgOut[i2,j2] = 1
        newPixels = newPixels + 1
    return newPixels

#adjacência-M
def adjM(i, j, img,newPixels):
    newPixels = adj4(i, j, img, newPixels)
    if(imgOut[i-1,j+1] == 0 and img[i-1,j+1] == 1):
        newPixels = interc(i, j, i-1, j+1, img, newPixels)
    if(imgOut[i-1,j-1] == 0 and img[i-1,j-1] == 1):
        newPixels = interc(i, j, i-1, j-1, img, newPixels)
    if(imgOut[i+1,j-1] == 0 and img[i+1,j-1] == 1):
        newPixels = interc(i, j, i+1, j-1, img, newPixels)
    if(imgOut[i+1,j+1] == 0 and img[i+1,j+1] == 1):
        newPixels = interc(i, j, i+1, j+1, img, newPixels)
    return newPixels
                
# detectar regiões de conexão
def connectRegions(img, x, y, tipo):
    global imgOut
    imgOut = np.zeros(np.shape(img))
    imgOut[x,y] = 1
    newPixels = 101
    newPixels2 = 0
    while newPixels>0:
        newPixels = 0
        newPixels2 = 0
        for i in range(1, np.shape(img)[0]-1):
            for j in range(1, np.shape(img)[1]-1):
                if imgOut[i,j] == 1:
                    if tipo == "adj4":
                        newPixels = newPixels + adj4(i, j, img, newPixels2)
                    elif tipo == "adj8":
                        newPixels = newPixels + adj8(i, j, img, newPixels2)
                    elif tipo == "adjM":
                        newPixels = newPixels + adjM(i, j, img, newPixels2)
                    
        for i in range(np.shape(img)[0]-2, 1, -1):
            for j in range(1, np.shape(img)[1]-1):
                if imgOut[i,j] == 1:
                    newPixels = newPixels + adjM(i,j,img,newPixels2)
    return imgOut

#função para pegar o ponto
def getPoint(event, x, y, flags, param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDOWN:
        mouseX,mouseY = y,x

In [29]:
# lendo a imagem
img = cv2.imread('test images/flower.jpg', cv2.IMREAD_COLOR)

#obtém a componente R
imgR = img[:,:,2]

#limiariza a imagem
imgT = threshold(imgR, 70)

#obter ponto na imagem
cv2.imshow('Threshold', imgT)
cv2.setMouseCallback('Threshold',getPoint)
cv2.waitKey(0)
cv2.destroyAllWindows()

#obtém região conectada
img4 = connectRegions(imgT, mouseX, mouseY, "adj4")
img8 = connectRegions(imgT, mouseX, mouseY, "adj8")
imgM = connectRegions(imgT, mouseX, mouseY, "adjM")

#calcular medidas
#widthD, heightD = metricsCalc(imgC, "DE")
#width4, height4 = metricsCalc(imgC, "D4")
#width8, height8 = metricsCalc(imgC, "D8")


#print('Euclidian Distance: width = ', widthD, 'height = ', heightD)
#print('City-block Distance: width = ', width4, 'height = ', height4)
#print('Chessboard Distance width = ', width8, 'height = ', height8)

# exibindo as imagens
cv2.imshow('Original Flower', img)
cv2.imshow('R Component', imgR)
cv2.imshow('Threshold', imgT)
cv2.imshow('Conect_4', img4)
cv2.imshow('Conect_8', img8)
cv2.imshow('Conect_M', imgM)
cv2.waitKey(0)

# limpando a memória
cv2.destroyAllWindows()

In [87]:
cv2.destroyAllWindows()

In [102]:
ph1, ph2, pw1, pw2 = (100, 112), (100, 149), (56, 136), (263, 136)

heightD = math.sqrt((ph2[0] - ph1[0])**2 + (ph2[1] - ph1[1])**2)
widthD  = math.sqrt((pw2[0] - pw1[0])**2 + (pw2[1] - pw1[1])**2)

height4 = abs(ph2[0] - ph1[0]) + abs(ph2[1] - ph1[1])
width4 = abs(pw2[0] - pw1[0]) + abs(pw2[1] - pw1[1])

height8 = max(abs(ph2[0] - ph1[0]), abs(ph2[1] - ph1[1]))
width8 =  max(abs(pw2[0] - pw1[0]), abs(pw2[1] - pw1[1]))


print('Euclidian Distance: width = ', widthD, 'height = ', heightD)
print('City-block Distance: width = ', width4, 'height = ', height4)
print('Chessboard Distance width = ', width8, 'height = ', height8)

Euclidian Distance: width =  207.0 height =  37.0
City-block Distance: width =  207 height =  37
Chessboard Distance width =  207 height =  37
